In [1]:
import numpy as numpy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random

In [2]:
articles = []
fakeArticles = []

In [3]:
fakeNews = []
fakeInput = open('fakeArticles.txt')
fakeArticles = fakeInput.readlines()
fakeInput.close

for i in range(len(fakeArticles)):
  if i%2 == 0:
    fakeNews.append(fakeArticles[i])
print(len(fakeNews))

323


In [4]:
realNews =[]
fileInput = open('articles.txt')
articles = fileInput.readlines()
fileInput.close

for i in range(len(fakeArticles)):
  if i%2 == 0:
    realNews.append(articles[i])
print(len(realNews))

323


In [5]:
T = int(len(realNews)*0.8)
trainNews = realNews[:T]+fakeNews[:T]
allNews = realNews+fakeNews

In [6]:
tokenifyer = Tokenizer(num_words=1000)
tokenifyer.fit_on_texts(trainNews)
testToken = tokenifyer.texts_to_sequences(trainNews)

In [7]:
tokenator = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenator.fit_on_texts(allNews)

In [8]:
allToken = tokenator.texts_to_sequences(allNews)
print(len(testToken[0]))
print(len(allToken[0]))

326
431


In [9]:
allToken = pad_sequences(allToken, padding='post')
maxL = len(allToken[0])

realLabels = [0]*len(realNews)
fakeLabels = [1]*len(fakeNews)
allLabels = realLabels+fakeLabels

labeledData = list(zip(allToken, allLabels))
random.shuffle(labeledData)
data = []
labels = []
data, labels = zip(*labeledData)

In [10]:
n = int(numpy.floor(len(data)*0.8))
trainingData = data[:n]
testingData = data[n:]

trainingLabels = labels[:n]
testingLabels = labels[n:]

In [11]:
trainingData = tf.constant(data[:n])
testingData = tf.constant(data[n:])

trainingLabels = tf.constant(labels[:n])
testingLabels = tf.constant(labels[n:])

In [12]:
model = keras.Sequential([
    keras.layers.Embedding(1000, 32, input_length=maxL),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
)

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 890, 32)           32000     
                                                                 
 global_average_pooling1d (G  (None, 32)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 100)               3300      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 24)                1224      
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                        

In [14]:
model.fit(trainingData, trainingLabels, epochs=20)

Epoch 1/20
17/17 [==============================] - 3s 13ms/step - loss: 0.6931 - accuracy: 0.5252
Epoch 2/20
17/17 [==============================] - 0s 15ms/step - loss: 0.6915 - accuracy: 0.6143
Epoch 3/20
17/17 [==============================] - 0s 13ms/step - loss: 0.6866 - accuracy: 0.8992
Epoch 4/20
17/17 [==============================] - 0s 13ms/step - loss: 0.6716 - accuracy: 0.9283
Epoch 5/20
17/17 [==============================] - 0s 13ms/step - loss: 0.6337 - accuracy: 0.8779
Epoch 6/20
17/17 [==============================] - 0s 13ms/step - loss: 0.5332 - accuracy: 0.9632
Epoch 7/20
17/17 [==============================] - 0s 11ms/step - loss: 0.3453 - accuracy: 0.9690
Epoch 8/20
17/17 [==============================] - 0s 13ms/step - loss: 0.1799 - accuracy: 0.9709
Epoch 9/20
17/17 [==============================] - 0s 12ms/step - loss: 0.0876 - accuracy: 0.9864
Epoch 10/20
17/17 [==============================] - 0s 12ms/step - loss: 0.0464 - accuracy: 0.9942
Epoch 11/

In [15]:
model.evaluate(testingData, testingLabels)

5/5 [==============================] - 0s 6ms/step - loss: 0.0872 - accuracy: 0.9692


[0.08719566464424133, 0.9692307710647583]

In [17]:
#model.predict(testingData)